## Deliverable 2. Create a Customer Travel Destinations Map.

In [62]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\Artsy\Desktop\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Jamestown,US,42.0970,-79.2353,65.55,91,95,9.64,overcast clouds
1,1,Brooks,CA,50.5834,-111.8851,78.78,46,9,4.61,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,41.49,66,63,1.36,broken clouds
3,3,Chokurdakh,RU,70.6333,147.9167,61.25,41,88,11.99,overcast clouds
4,4,Yulara,AU,-25.2406,130.9889,64.90,28,0,16.04,clear sky


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temp value

min_temp = float(input("What is the lowest temperature(F) you would like for your trip "))
max_temp = float(input("What is the highest temperature(F) you would like for your trip "))

What is the lowest temperature(F) you would like for your trip 60
What is the highest temperature(F) you would like for your trip 80


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Jamestown,US,42.0970,-79.2353,65.55,91,95,9.64,overcast clouds
1,1,Brooks,CA,50.5834,-111.8851,78.78,46,9,4.61,clear sky
3,3,Chokurdakh,RU,70.6333,147.9167,61.25,41,88,11.99,overcast clouds
4,4,Yulara,AU,-25.2406,130.9889,64.90,28,0,16.04,clear sky
6,6,Port Macquarie,AU,-31.4333,152.9167,61.90,72,82,8.08,broken clouds


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [57]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Jamestown,US,65.55,overcast clouds,42.0970,-79.2353,
1,Brooks,CA,78.78,clear sky,50.5834,-111.8851,
3,Chokurdakh,RU,61.25,overcast clouds,70.6333,147.9167,
4,Yulara,AU,64.90,clear sky,-25.2406,130.9889,
6,Port Macquarie,AU,61.90,broken clouds,-31.4333,152.9167,


In [60]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [63]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #try:
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("End of Sequence")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


KeyboardInterrupt: 

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
